In [5]:
!pip install pandas
!pip install scikit-learn
!pip install openpyxl
!pip install tensorflow
!pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/37/19/0423e5e1fd1c6ec5be2352ba05a537a473c1677f8188b9306097d684b327/scikit_learn-1.6.1-cp310-cp310-macosx_12_0_arm64.whl.metadata
  Using cached scikit_learn-1.6.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (31 kB)
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/ea/b1/4deb37252311c1acff7f101f6453f0440794f51b6eacb1aad4459a134081/scipy-1.15.3-cp310-cp310-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/da/d3/13ee227a148af1c693654932b8b0b02ed64af5e1f7406d56b088b57574cd/joblib-1.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https:/

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam as KAdam, RMSprop, SGD as KSGD

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Configurações

In [5]:
features = [
    'PRN',
    'Carrier_Doppler_hz',
    'Pseudorange_m',
    'RX_time',
    'TOW_at_current_symbol_s',
    'Carrier_phase_cycles',
    'CN0',
]

solvers = {
    'Levenberg–Marquardt (lbfgs)': {
        'solver': 'lbfgs'
    },
    'Adam': {
        'solver': 'adam',
        'learning_rate_init': 0.001
    },
    'SGD': {
        'solver': 'sgd',
        'learning_rate_init': 0.01
    },
}

### 1 Tratamento dos dados

In [ ]:
df = pd.read_excel(
    os.getenv("DATASET_PATH_AUTHENTIC_AND_SIMPLIFIED"),
    sheet_name=0,
    header=[0,1]
)

### 1.1 Desempilhar canais

In [7]:
rows = []
for ch in df.columns.levels[1]:
    X_ch = df[features].xs(ch, axis=1, level=1)
    X_ch.columns = features
    y_ch = df['Output'][ch].rename('label')
    rows.append(pd.concat([X_ch, y_ch], axis=1))

long_df = pd.concat(rows, ignore_index=True)

### 1.2 Separar features e labels

In [ ]:
X = long_df[features].values
y = long_df['label'].astype(int).values

### 1.3 Split + normalização

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

## 2 Comparar solvers

In [ ]:
for name, params in solvers.items():
    mlp = MLPClassifier(
        hidden_layer_sizes=(64,32),
        activation='relu',
        alpha=1e-4,
        batch_size=128,
        max_iter=60,
        random_state=42,
        verbose=False,
        **params
    )
    mlp.fit(X_train, y_train)
    acc = accuracy_score(y_test, mlp.predict(X_test))
    print(f'{name:<35} → Test Accuracy: {acc:.4f}')

## 3 Experimentar RMSProp via Keras

In [ ]:
def eval_keras(optimizer, label):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1,  activation='sigmoid'),
    ])
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    model.fit(
        X_train,
        y_train,
        epochs=50,
        batch_size=128,
        verbose=0
    )
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f'{label:<35} → Test Accuracy: {acc:.4f}')

eval_keras(KAdam(learning_rate=0.001), 'Adam')
eval_keras(RMSprop(learning_rate=0.001), 'RMSProp')
eval_keras(KSGD(learning_rate=0.01), 'SGD')